# Predección del semestre en cual un estudiante abandona sus estudios de pregrado

<img src="logo_udea.png" alt="drawing" width="100"/>



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pandas import read_excel
from sklearn.model_selection import train_test_split

## Leyendo los datos

In [2]:
my_sheet = 'SinDuplicadosDepurada' # change it to your sheet name
file_name = 'BDFinal.xlsx' # change it to the name of your excel file
df = read_excel(file_name, sheet_name=my_sheet)
df.shape

(6593, 64)

En la siguiente fila se filtra por sede y modalidad.

In [3]:
df = df[(df.SEDE == 'MEDELLIN') & (df.MODALIDAD == 'PRESENCIAL')]


En la siguiente línea se crea la variable `y` categórica.

In [4]:
df['y'] = pd.Categorical(df['NIVEL_PREGRADO'])

Para ver una tabla de frecuencia de los diferentes valores de `y` usamos la siguiente instrucción:

In [5]:
pd.crosstab(index=df['y'], columns="Frecuencia")

col_0,Frecuencia
y,
1,3482
2,714
3,384
4,199
5,117
6,61
7,24
8,9
9,5


In [6]:
dummy_cuali_data = pd.get_dummies(df[['SEXO', 'RANGO', 'cole_genero']], drop_first=True)

In [7]:
num_data = df[['PROMEDIO_ULTIMO_SEMESTRE_TERMINADO', 
               'VERSION',
               'CREDITOS_ULTIM_SEMEST_MATRIC',
               'IND_POBREZA',
               'Puntaje_total_estan',
               'Puntaje_global',
               'estu_edad']]

covariables = pd.concat([dummy_cuali_data, num_data], axis=1)

## Creación de los datos de entrenamiento (train) y de validación (test)
Para particionar los datos originales se usa la función `train_test_split`, para mayores detalles se recomienda consultar los parámetros de la función se recomienda consultar este [enlace](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(covariables, df['y'], test_size=0.20)

## Ajuste del modelo SVM

Para crear el modelo se usa la función `sklearn.svm.SVC`, para mayores detalles se recomienda consultar los parámetros de la función se recomienda consultar este [enlace](https://scikit-learn.org/0.16/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

In [9]:
cw = {}
for i in set(y_train):
    cw[i] = np.sum(y_train == i)
print(cw)

{1: 2795, 2: 567, 3: 303, 4: 163, 5: 85, 6: 47, 7: 20, 8: 8, 9: 4, 10: 17, 11: 6}


In [10]:
from sklearn.svm import SVC

model = SVC(class_weight=cw, coef0=0.0, degree=3, gamma='auto', kernel='linear')
model_fit = model.fit(X=X_train, y=y_train)

Para obtener la tabla de confusión hacemos lo siguiente:

In [11]:
t1 = pd.crosstab(index=model.predict(X_train), columns=y_train, rownames=['Prediccion'], colnames=['Membresia'])
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])

print('\n')
print('La tabla de confusión usando train es: \n')
print(t1)

print('\n')
print('La tabla de confusión usando test es: \n')
print(t2)



La tabla de confusión usando train es: 

Membresia     1    2    3    4   5   6   7   8   9   10  11
Prediccion                                                 
1           2795  567  303  163  85  47  20   8   4  17   6


La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10
Prediccion                                          
1           687  147  81  36  32  14   4   1   1   1


In [12]:
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))

El Accuracy con test es: 0.6961394769613948
El Accuracy con train es: 0.6842629482071713


## Ajuste del modelo KNN

In [13]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=10)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10
Prediccion                                          
1           677  144  79  35  32  13   4   1   1   1
2             8    2   2   1   0   1   0   0   0   0
3             2    1   0   0   0   0   0   0   0   0


El Accuracy con test es: 0.7001245330012453
El Accuracy con train es: 0.6762948207171314


## Ajuste del modelo ARBOL

In [14]:
from sklearn import tree

model = tree.DecisionTreeClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1   2   3   4   5   6   7   8   9   10
Prediccion                                         
1           500  94  44  18  14   5   1   1   1   0
2            89  23  15   4   7   3   1   0   0   1
3            45  17   9   6   6   2   1   0   0   0
4            25   6   7   3   2   2   1   0   0   0
5            14   4   0   2   0   2   0   0   0   0
6             7   1   2   2   1   0   0   0   0   0
7             3   1   1   0   0   0   0   0   0   0
8             0   1   1   1   0   0   0   0   0   0
10            3   0   2   0   2   0   0   0   0   0
11            1   0   0   0   0   0   0   0   0   0


El Accuracy con test es: 1.0
El Accuracy con train es: 0.5328685258964143


## Ajuste del modelo Gradient Tree Boosting

In [15]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10
Prediccion                                          
1           666  141  78  32  30  12   4   1   1   1
2             7    2   0   1   0   0   0   0   0   0
3             2    2   1   1   1   1   0   0   0   0
4             5    1   1   1   0   1   0   0   0   0
5             1    0   0   0   0   0   0   0   0   0
6             0    1   0   0   0   0   0   0   0   0
7             2    0   0   0   0   0   0   0   0   0
8             2    0   0   0   0   0   0   0   0   0
9             1    0   0   0   0   0   0   0   0   0
10            1    0   1   1   0   0   0   0   0   0
11            0    0   0   0   1   0   0   0   0   0


El Accuracy con test es: 0.7723536737235367
El Accuracy con train es: 0.6673306772908366


## Ajuste del modelo Forests of randomized trees¶

In [20]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1   2   3   4   5   6   7   8   9   10
Prediccion                                         
1           505  95  42  17  13   5   1   1   1   0
2            86  25  17   6   9   3   1   0   0   1
3            49  14   9   6   3   2   1   0   0   0
4            21   6   5   3   3   3   1   0   0   0
5            13   4   1   2   0   1   0   0   0   0
6             7   1   2   1   1   0   0   0   0   0
7             2   1   1   0   0   0   0   0   0   0
8             0   0   1   1   0   0   0   0   0   0
9             0   1   0   0   1   0   0   0   0   0
10            3   0   3   0   2   0   0   0   0   0
11            1   0   0   0   0   0   0   0   0   0


El Accuracy con test es: 1.0
El Accuracy con train es: 0.5398406374501992


## Ajuste del modelo Multioutput classification

In [17]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 100, 100), max_iter=300)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10
Prediccion                                          
1           687  147  81  36  32  14   4   1   1   1


El Accuracy con test es: 0.6961394769613948
El Accuracy con train es: 0.6842629482071713


## Ajuste del modelo Stochastic Gradient Descent

In [18]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(loss="hinge", penalty="l2", max_iter=5000)
model_fit = model.fit(X=X_train, y=y_train)

print('\n')
t2 = pd.crosstab(index=model.predict(X_test), columns=y_test, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
print('El Accuracy con test es:', model.score(X_train, y_train))
print('El Accuracy con train es:', model.score(X_test, y_test))



La tabla de confusión usando test es: 

Membresia    1    2   3   4   5   6   7   8   9   10
Prediccion                                          
1           139   14   5   2   5   2   0   0   0   0
2           548  133  76  34  27  12   4   1   1   1


El Accuracy con test es: 0.2921544209215442
El Accuracy con train es: 0.27091633466135456


## Un modelo adivinando

In [19]:
respuesta = pd.crosstab(index=df['y'], columns="Frecuencia")

import random

n = X_train.shape[0] # numero observaciones en train
y_al_azar = random.choices(population=respuesta.index.values, weights=respuesta['Frecuencia'], k=n)
y_al_azar = pd.Series(y_al_azar)

print('\n')
t2 = pd.crosstab(index=y_al_azar, columns=y_train, rownames=['Prediccion'], colnames=['Membresia'])
print('La tabla de confusión usando test es: \n')
print(t2)

print('\n')
accu = t2.values.diagonal().sum() / t2.values.sum()
print('El Accuracy con train es:', accu)



La tabla de confusión usando test es: 

Membresia     1    2    3   4   5   6   7   8   9   10  11
Prediccion                                                
1           1079  231  150  90  50  29  15   1   4   5   4
2            200   52   22  16   8   7   2   1   0   2   0
3            110   29   14   9   5   2   0   2   0   2   1
4             63   14    7   9   7   3   1   1   0   1   1
5             29   11    9   4   2   1   0   1   0   2   0
6             19    3    4   0   1   1   0   0   0   0   0
7              7    4    1   0   0   0   0   0   0   0   0
8              3    1    0   1   0   0   0   0   0   0   0
9              1    0    0   0   0   0   0   0   0   0   0
10             5    1    0   0   0   0   0   0   0   0   0
11             4    2    1   1   0   0   0   0   0   0   0


El Accuracy con train es: 0.48859797297297297
